# Advent of Code 2020
Basic imports of stuff

In [1]:
from aocd import submit
from aocd.models import Puzzle, User
from pathlib import Path
from itertools import islice, combinations
from functools import reduce
from collections import deque
import re

current_day is only available in December (EST)


Useful stuff

In [2]:
def first(iterable):
    '''returns the first item of an iterable'''
    return next(iter(iterable))

def nth(iterable, n, default=None):
    '''Returns the nth item in an iterable or a default value'''
    return next(islice(iterable, n, None), default)

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def tail(n, iterable):
    "Return an iterator over the last n items"
    # tail(3, 'ABCDEFG') --> E F G
    return iter(deque(iterable, maxlen=n))

def prod(numbers):
    '''Returns the product of all the numbers'''
    return reduce((lambda x,y: x*y), numbers)


## Day 01
### Part 1
The puzzle input is a list of numbers (provided as a long string, separated by newlines).
I need to fine the 2 entries that sum to 2020, then provide the multiple of the 2 entries


In [3]:
p = Puzzle(year=2020, day=1)
d = [int(x) for x in p.input_data.split()]

In [4]:
result = None
for entries in combinations(d,2):
    if 2020 == sum(entries):
        result = prod(entries)
        break

if result:
    print(result)
    p.answer_a = result

960075


In [8]:
prod(first(entry for entry in combinations(d,2) if sum(entry) == 2020))

960075

### Part 2
Same puzzle but find the combination of 3 entries that add up to 2020

In [5]:
result = None
for entries in combinations(d,3):
    if 2020 == sum(entries):
        result = prod(entries)
        break

if result:
    print(result)
    p.answer_b = result

212900130


In [9]:
prod(first(entry for entry in combinations(d,3) if sum(entry)==2020))

212900130

## Day 02
### Part 1
Validate passwords in a list (puzzle input) using rules associated with each password.  Using regex to make this work.


In [6]:
p = Puzzle(year=2020, day=2)


In [7]:
result = 0

parser = re.compile('(\d*)-(\d*).(\w): (\w*)')

for lo, hi, letter, password in parser.findall(p.input_data):
    if int(lo) <= password.count(letter) <= int(hi):
        result+=1

print(result)
p.answer_a = result

422


### Part 2
This time the numbers tell you which indexes to check.  Only one of the indexes can contain the specified letter

In [8]:
result = 0

parser = re.compile('(\d*)-(\d*).(\w): (\w*)')

for lo, hi, letter, password in parser.findall(p.input_data):
    if int(lo) < int(hi) <= len(password):
        if (password[int(lo)-1] == letter) ^ (password[int(hi)-1] == letter):
            result+=1

print(result)
p.answer_b = result

451


## Day 3
### Part 1


In [9]:
p = Puzzle(day=3,year=2020)

In [10]:
d = p.input_data.splitlines()

In [11]:
result = 0

start = (0,0)
slope = (3,1)
ncol = len(d[0])

here = start
while here[1] < len(d):
    if d[here[1]][here[0]] == '#':
        result += 1
    here = (here[0] + slope[0]) % ncol , here[1] + slope[1]

p.answer_a= result
 

### Part 2

In [12]:
results = []

start = (0,0)
ncol = len(d[0])

slopes = [(1,1), (3,1), (5,1), (7,1), (1,2)]

for slope in slopes:
    here = start
    result = 0
    while here[1] < len(d):
        if d[here[1]][here[0]] == '#':
            result += 1
        here = (here[0] + slope[0]) % ncol , here[1] + slope[1]
    
    results.append(result)

p.answer_b = prod(results)

## Day 4
### Part 1

In [13]:
p = Puzzle(day=4, year=2020)

In [14]:

parser = re.compile('(\w{3}):(\S+)')

def has_all_fields(passport):
    required_fields = set(['byr','iyr','eyr','hgt','hcl','ecl','pid'])
    detected_fields = set(field[0] for field in parser.findall(passport))
    return not(required_fields - detected_fields)

passports = p.input_data.split('\n\n')
valid_passports = [passport for passport in passports if has_all_fields(passport)]

p.answer_a = len(valid_passports)

### Part 2
The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

 - byr (Birth Year) - four digits; at least 1920 and at most 2002.
 - iyr (Issue Year) - four digits; at least 2010 and at most 2020.
 - eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
 - hgt (Height) - a number followed by either cm or in:
 - If cm, the number must be at least 150 and at most 193.
 - If in, the number must be at least 59 and at most 76.
 - hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
 - ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
 - pid (Passport ID) - a nine-digit number, including leading zeroes.
 - cid (Country ID) - ignored, missing or not.

In [15]:
def validate_byr(year: str) -> bool:
    return 1920 <= int(year) <= 2002

def validate_iyr(year: str):
    return 2010 <= int(year) <= 2020

def validate_eyr(year: str): 
    return 2020 <= int(year) <= 2030

def validate_hgt(height: str):
    height = height.strip()

    try:
        height, unit = int(height[:-2]), height[-2:]
    except(ValueError):
        return False

    if(unit == 'cm'):
        return 150 <= height <= 193
    elif(unit == 'in'):
        return 59 <= height <= 76
    else: 
        return False

def validate_hcl(color: str):
    if color[0] != '#':
        return False

    if len(color[1:]) != 6:
        return False

    try:
        color = int(color[1:],16)
    except(ValueError):
        return False

    return True

def validate_ecl(color: str):
    valid_colors = ['amb', 'blu', 'brn','gry', 'grn', 'hzl', 'oth']
    return color in valid_colors

def validate_pid(pid: str):
    if(len(pid) != 9):
        return False

    try:
        pid = int(pid)
    except(ValueError):
        return False

    return True


In [16]:
validator = {'byr': validate_byr,
             'iyr': validate_iyr,
             'eyr': validate_eyr,
             'hgt': validate_hgt,
             'hcl': validate_hcl,
             'ecl': validate_ecl,
             'pid': validate_pid,
             'cid': lambda x: True}

passports = p.input_data.split('\n\n')
valid_passports = []
for passport in passports:
    if has_all_fields(passport):
        is_valid = True
        for field, val in parser.findall(passport):
            is_valid = is_valid and validator[field](val)
        
        if is_valid:
            valid_passports.append(passport)

p.answer_b = len(valid_passports)

## Day 5
### Part 1
 - F = 0
 - B = 1
 - L = 0
 - R = 1      

In [17]:
p = Puzzle(year = 2020, day=5)

In [18]:
seat_ids = p.input_data.splitlines()

table = str.maketrans('FBLR','0101')

def toBinaryStr(seat_id: str) -> str:
    return seat_id.translate(table)

def inDecimal(seat_id: str) -> int:
    return int(toBinaryStr(seat_id), 2)

maxSeat = max(inDecimal(seat) for seat in seat_ids)
p.answer_a = maxSeat


### Part 2

In [19]:
seats = [inDecimal(seat_id) for seat_id in seat_ids]
seats.sort()

diffs = [b-a for a,b in zip(seats[:-1],seats[1:])]

my_seat = seats[diffs.index(2)] + 1 

p.answer_b = my_seat